설정 / 앱 / vscode, Miniconda3, python을 제거
사용자계정 폴더 .conda .ipython .vscode .miniconda3 폴더외, condarc 파일 제거
C:\User

In [2]:
%pip install langchain langchain-openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import load_dotenv
import os
load_dotenv()
os.getenv('OPENAI_API_KEY')[:4]

'sk-p'

Langchain의 핵심 컴포넌트 : 모델 호출계를 구성하는 추상화 요소를 제공
```
    - PromptTemplate : LLM에 보낼 입력 프롬프트
    - ChatOpenAI : openai의 GPT - 모델 호출
    - Runnable : 실행가능한 객체에 대한 공통 인터페이스 -> invoke() 매서드를 통해서 입력과 출력을 지원
    - StrOutPutParser : 문자열 출력 파서
파이프로 연결 가능... ex) prompt | llm | strparser
```

In [4]:
from langchain_core.prompts import PromptTemplate
template = '{product}제품을 생산하는 회사이름은 뭘로 하면 좋을까?'
prompt = PromptTemplate.from_template(template)
formated_prompt = prompt.format(product = '커피')
formated_prompt

'커피제품을 생산하는 회사이름은 뭘로 하면 좋을까?'

In [5]:
from langchain_openai import ChatOpenAI
# 위에서 선언했기 때문에 key값을 입력하지 않아도 됨
llm=ChatOpenAI(model='gpt-4o-mini', temperature=0) # Runnable 객체 --> invoke()
response = llm.invoke([('human','안녕')])
response

AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 9, 'total_tokens': 19, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BbGg2MWS8ekVWC4EO16lMHli4UAuU', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--1ab0df0e-8b72-46de-aa72-67ed04891e26-0', usage_metadata={'input_tokens': 9, 'output_tokens': 10, 'total_tokens': 19, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [6]:
# 문자열 출력 파서 Runnable 객체 -invoke
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser_text= parser.invoke(response)
parser_text

'안녕하세요! 어떻게 도와드릴까요?'

LangChain Expression Language(LCEL) 단일 체인 실행

In [7]:
chain = prompt | llm | parser  # runnable 객체 --> invoke
result = chain.invoke(["product","coffee"])
print(result)

커피 제품을 생산하는 회사 이름으로는 다음과 같은 아이디어를 고려해볼 수 있습니다:

1. **커피의 정원** (Garden of Coffee)
2. **아침의 향기** (Morning Aroma)
3. **커피의 예술** (Art of Coffee)
4. **커피 마법** (Coffee Magic)
5. **한 잔의 행복** (Cup of Joy)
6. **커피 이야기** (Coffee Tales)
7. **커피의 여정** (Journey of Coffee)
8. **프리미엄 로스트** (Premium Roast)
9. **커피의 미소** (Coffee Smile)
10. **커피의 순간** (Moments of Coffee)

이름은 브랜드의 이미지와 목표에 맞게 선택하는 것이 중요하니, 여러 가지 아이디어를 조합해보거나 변형해보는 것도 좋습니다!


Langchain을 활용한 모델 사용, 비용 모니터링 및 개성 전략
```
Langchain을 활용한 모델 사용, 비용모니터링 및 캐싱 전략
    GPT-4o-mini GPT 3.5-Turbo 비용이 60% 저렴
    Langchain v0.3부터 openAI가 별도 패키지로 분리 필요 패키지를 설치 langchain-openai 필요
    토큰 사용량 추정, 캐싱을 위한 LangChain-community도 별도 설치
    환경변수 변수 관리 패키지 python-dotevn
```

In [8]:
%pip install langchain-openai langchain-cummunity python-dotenv openai

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement langchain-cummunity (from versions: none)
ERROR: No matching distribution found for langchain-cummunity


In [9]:
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.7)
prompt = 'LangChain에 대해 한문장으로 설명해줘'
result = llm.invoke(prompt)
result.content

'LangChain은 다양한 언어 모델과 데이터 소스를 연결하여 자연어 처리 애플리케이션을 쉽게 구축할 수 있도록 지원하는 프레임워크입니다.'

In [11]:
# 사용량
result.usage_metadata

{'input_tokens': 18,
 'output_tokens': 35,
 'total_tokens': 53,
 'input_token_details': {'audio': 0, 'cache_read': 0},
 'output_token_details': {'audio': 0, 'reasoning': 0}}

In [12]:
# 콜백함수를 통한 누적 토큰 추적(get_openai_callback)
from langchain_community.callbacks import get_openai_callback
with get_openai_callback() as cb:
    # 첫번째 호출
    res1 = llm.invoke('서울의 오늘 날씨는 어떤지 알려줘')
    print('응답1', res1.content[:10], '...')
    # 두번째 호출
    res2 = llm.invoke('파이썬으로 랭체인 사용법 알려줘')
    print('응답2', res2.content[:10], '...')

# 누적 토큰 사용량 출력 콜백 cb에는 블록 전체 토큰 사용량이 누적
# 총 토큰수
print('총 토큰수 : ',cb.total_tokens)
# 프롬프트 토큰수
print('프롬프트 토큰수 : ', cb.prompt_tokens)
# 응답 토큰수
print('응답토큰수 : ', cb.completion_tokens)
# 비용계산
print('비용계산(USD) : ', cb.total_cost)

응답1 죄송하지만, 실시간 ...
응답2 랭체인(RLangC ...
총 토큰수 :  737
프롬프트 토큰수 :  37
응답토큰수 :  700
비용계산(USD) :  0.00042555


In [15]:
# LangChain 의 LLM 응답캐싱 (InMemorry Cache, SQLiteCache)
# 동일한 질문은 저장해 뒀다가 응답에 사용
from langchain_core.caches import InMemoryCache
from langchain_core.globals import set_llm_cache
# InMemorryCache 설정
set_llm_cache(InMemoryCache())

In [23]:
# 캐시 사용 전후를 비교, 같은 질문을 두번 호출
query = '재미있는 유머 하나 알려줘'
# 첫 번째 호출 (캐시에 없으면 api 호출 발생)
result1 = llm.invoke(query)
print(f'응답1 : {result1.content}')
print('-'*50)
# 두 번째 호출(동일한 query, 캐서를 확인하고 동일 질문하면 api 미호출)
result2 = llm.invoke(query)
print(f'응답2 : {result2.content}')
print('-'*50)
query = query + '?'
# 세 번째 호출(!가 추가된 query)
result3 = llm.invoke(query)
print(f'응답3 : {result3.content}')

응답1 : 물고기가 학교에 가면 뭐라고 부를까요?

"피시 스쿨!" 🐟😄
--------------------------------------------------
응답2 : 물고기가 학교에 가면 뭐라고 부를까요?

"피시 스쿨!" 🐟😄
--------------------------------------------------
응답3 : 물론이죠! 

왜 컴퓨터는 바다를 싫어할까요?

바다에 가면 항상 "버그"가 생기니까요! 🐛💻

재미있으셨나요? 다른 유머도 원하시면 말씀해 주세요!


In [29]:
# 실행시간 측정
import time
# 첫 번째 호출 시간
query = '오늘 점심 메뉴 추천 해줘!'
start = time.time(); llm.invoke(query); end = time.time()
print(f'첫번째 호출시간 : {end-start}')

start = time.time(); llm.invoke(query); end = time.time()
print(f'두번째 호출시간 : {end-start}')

# InMemorry 방식은 어플리케이션을 종료하면 저장된 내용이 날라간다

첫번째 호출시간 : 4.591396808624268
두번째 호출시간 : 0.0009007453918457031


In [ ]:
# 임시로 In-Memory 캐시로 설정하여 DB 연결을 끊기
# set_llm_cache(InMemoryCache())


# SQLite 캐시(디스크기반 캐시)
import os, time
from langchain_community.cache import SQLiteCache
# 기존 캐시 DB 삭제(.langchain.db 초기화)

# if os.path.exists('.langchain.db'):
#     os.remove('.langchain.db')

# SQLite 캐시 설정(지정한 경로의 DB 파일을 생성 / 사용)
set_llm_cache(SQLiteCache(database_path = '.langchain.db')) # langchain.db 있으면 사용 없으면 생성

# 동일한 query를 두면 호출해서 결과와 시간을 비교
query = '저녁메뉴 추천해줘'
# 첫 번째 호출(캐시에 없으면 api 호출 발생)
start = time.time(); result1 = llm.invoke(query); end = time.time()
print(f'첫 번째 호출시간 : {end-start}')
print(f'응답1 : {result1.content}')

# 두 번째 호출(캐시에 없으면 api 호출 발생)
start = time.time(); result2 = llm.invoke(query); end = time.time()
print(f'두 번째 호출시간 : {end-start}')
print(f'응답2 : {result2.content}')


첫 번째 호출시간 : 4.121056795120239
응답1 : 물론이죠! 저녁 메뉴로 몇 가지 추천해드릴게요.

1. **된장찌개와 밥**: 한국의 전통적인 된장찌개에 밥을 곁들여 간단하면서도 맛있는 한 끼를 즐길 수 있습니다.

2. **불고기**: 양념된 소고기를 볶아내어 쌈채소와 함께 먹으면 훌륭한 저녁이 됩니다.

3. **잡채**: 당면과 다양한 채소, 고기를 볶아 만든 잡채는 맛도 좋고 영양도 풍부합니다.

4. **해물파전**: 비 오는 날에 특히 잘 어울리는 해물파전은 간단하게 만들 수 있고, 막걸리와 함께하면 더욱 좋습니다.

5. **치킨**: 바삭하게 튀긴 치킨에 다양한 사이드 메뉴를 곁들여 즐길 수 있는 저녁도 좋습니다.

어떤 메뉴가 마음에 드세요? 추가로 다른 아이디어가 필요하면 말씀해 주세요!
두 번째 호출시간 : 0.0030121803283691406
응답2 : 물론이죠! 저녁 메뉴로 몇 가지 추천해드릴게요.

1. **된장찌개와 밥**: 한국의 전통적인 된장찌개에 밥을 곁들여 간단하면서도 맛있는 한 끼를 즐길 수 있습니다.

2. **불고기**: 양념된 소고기를 볶아내어 쌈채소와 함께 먹으면 훌륭한 저녁이 됩니다.

3. **잡채**: 당면과 다양한 채소, 고기를 볶아 만든 잡채는 맛도 좋고 영양도 풍부합니다.

4. **해물파전**: 비 오는 날에 특히 잘 어울리는 해물파전은 간단하게 만들 수 있고, 막걸리와 함께하면 더욱 좋습니다.

5. **치킨**: 바삭하게 튀긴 치킨에 다양한 사이드 메뉴를 곁들여 즐길 수 있는 저녁도 좋습니다.

어떤 메뉴가 마음에 드세요? 추가로 다른 아이디어가 필요하면 말씀해 주세요!


LangChain 프롬프트 템플릿
```
LLM 프롬프트를 동적으로 구성하고 재 사용할 수 있도록 해주는 도구
    단일입력 : 하나의 변수로 구성된 프롬프트 템플릿
    다중입력 : 둘 이상의 변수를 사용하는 템플릿
    ChatPromptTemplate 역할 기반 프롬프트 : 시스템/사용처 역할별 프롬프트 구성 .from_massage()
    PartialPromptTemplate 활용 : 프롬프트 일부를 미리 고정하고 부분포멧팅을 사용해(ex 시스템 메세지는 고정...)
    프롬프트 출력 및 체인 실행 : LCEL
    프롬프트 작성 팁: 주의사항 및 모범사례
```

In [51]:
# 환경설정
%pip install langchain langchain-openai python-devenv --quiet

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement python-devenv (from versions: none)
ERROR: No matching distribution found for python-devenv


In [52]:
from dotenv import load_dotenv
load_dotenv()

True

In [54]:
# 단일 프롬프트 사용
from langchain.prompts import PromptTemplate
# 템플릿 문자열 정의
template_str = (
    "당신은 최고 수준의 마케팅 카피라이터 입니다.\n"
    "아래 제품의 매렺거인 홍부문굴르 100자 내외로 작성해 주세요\n\n"
    "제품명 : {product_name}"
)
# promptTemplate 객체 생성
product_prompt = PromptTemplate.from_template(template_str)
# 템플릿에 값 채우기
formated_prompt = product_prompt.format(product_name = '이클립스')
print(formated_prompt)

당신은 최고 수준의 마케팅 카피라이터 입니다.
아래 제품의 매렺거인 홍부문굴르 100자 내외로 작성해 주세요

제품명 : 이클립스


In [57]:
# 프롬프트 | llm --> invoke
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)
# Runnable 객체 생성  LCEL
chain = product_prompt | llm
result = chain.invoke({'product_name':'커피'})
print(result.content)

"하루의 시작을 특별하게! 깊고 풍부한 향의 커피 한 잔으로 당신의 감각을 깨워보세요. 매일의 일상에 작은 행복을 더하는 커피, 지금 바로 경험해보세요!"


In [63]:
# 다중입력
# 다중입력 템플릿 문자열 저의
multi_tmeplate_str = (
    "아래는 뉴스기사 제목과 키워드 입니다.\n"
    "이 정보를 바탕으로 한 문단으로 구성된 간락한 요약문을 작성하세요\n\n"
    "제목:{title}\n"
    "키워드:{keyword}"
)
# 프롬프트 템플릿 작성
summary_prompt = PromptTemplate(template=multi_tmeplate_str, input_variables=['title','keyword'])
# 포멧팅을 통해 프롬프트 값 확인
sample_title = '인공지능 기술의 발전과 미래'
sample_keyword = '머신러닝,딥러닝,LLM,랭체인,산업 혁신'
formatted_summary_prompt = summary_prompt.format(title=sample_title, keyword =sample_keyword)
print(formatted_summary_prompt)

아래는 뉴스기사 제목과 키워드 입니다.
이 정보를 바탕으로 한 문단으로 구성된 간락한 요약문을 작성하세요

제목:인공지능 기술의 발전과 미래
키워드:머신러닝,딥러닝,LLM,랭체인,산업 혁신


In [67]:
# LCEL 출력
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
result_summary = (summary_prompt | llm | parser).invoke({
    'title': sample_title,
    'keyword' : sample_keyword
})
result_summary

'인공지능 기술의 발전은 머신러닝과 딥러닝의 혁신을 통해 가속화되고 있으며, 특히 대규모 언어 모델(LLM)과 랭체인 기술이 주목받고 있습니다. 이러한 기술들은 다양한 산업 분야에서 혁신을 이끌어내고 있으며, 기업들은 이를 활용하여 효율성을 높이고 새로운 비즈니스 모델을 창출하고 있습니다. 앞으로 인공지능의 지속적인 발전은 우리의 삶과 산업 구조에 큰 변화를 가져올 것으로 기대됩니다.'

In [68]:
# ChatPromptTemplate과 역할 기반 프롬프트
# 시스템/사용자/어시스턴트 역할(role)
# 시스템 메시지 : 모델의 동작을 지시
# 사용자 메세지 : 실제 사용자의 입력
# 어시스턴트 메시지 : 이전 모델이 응답한 내용이 있으면 대화맥락유지를 위해 활용

In [73]:
from langchain.prompts import ChatPromptTemplate
system_message = ("당신은 python 분야의 뛰어난 전문가이자 조언자 입니다."
                  "사용자의 프로그래밍 질문에 대해 친절하고 이해하기 쉽게 단변해 주세요")
user_message = "{question}"
chat_prompt = ChatPromptTemplate.from_messages([
    ('system',system_message),
    ('user', user_message)
])

sample_question = '파이썬에 클래스에 대해 설명해주세요'
formated_massage = chat_prompt.format_messages(question=sample_question)
print(formated_massage)
# 파이프라인을 이용해서 llm호출 및 파싱
answer = (chat_prompt | llm | parser).invoke({'question': sample_question})
print(answer)

[SystemMessage(content='당신은 python 분야의 뛰어난 전문가이자 조언자 입니다.사용자의 프로그래밍 질문에 대해 친절하고 이해하기 쉽게 단변해 주세요', additional_kwargs={}, response_metadata={}), HumanMessage(content='파이썬에 클래스에 대해 설명해주세요', additional_kwargs={}, response_metadata={})]
파이썬에서 클래스는 객체 지향 프로그래밍(OOP)의 기본 개념 중 하나로, 객체를 생성하기 위한 청사진(템플릿)입니다. 클래스를 사용하면 관련된 데이터(속성)와 기능(메서드)을 하나의 단위로 묶을 수 있습니다.

### 클래스의 기본 구조

클래스를 정의하려면 `class` 키워드를 사용합니다. 다음은 간단한 클래스의 예입니다:

```python
class Dog:
    # 생성자: 객체가 생성될 때 호출되는 메서드
    def __init__(self, name, age):
        self.name = name  # 속성: 개의 이름
        self.age = age    # 속성: 개의 나이

    # 메서드: 개가 짖는 행동을 정의
    def bark(self):
        return f"{self.name} says woof!"

# 클래스 사용
my_dog = Dog("Buddy", 3)  # Dog 클래스의 인스턴스 생성
print(my_dog.bark())       # "Buddy says woof!" 출력
```

### 주요 개념

1. **속성 (Attributes)**: 클래스의 데이터. 위 예제에서 `name`과 `age`가 속성입니다.
2. **메서드 (Methods)**: 클래스의 기능. `bark` 메서드는 개가 짖는 행동을 정의합니다.
3. **생성자 (Constructor)**: `__init__` 메서드는 객체가 생성될 때 호출되어 초기 속성을 설정합니다.
4. **인스턴스 (Instanc

In [79]:
# ParitalPromptTemplate : 템플릿의 일부를 부분적으로 채운 새로운 템플릿
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate
role_system_template = '당신은 {role} 분야의 전문 지식인 입니다. 가능한 자세히 답변해 주세요'
system_prompt = SystemMessagePromptTemplate.from_template(role_system_template)
user_prompt = HumanMessagePromptTemplate.from_template("{question}")

# chatprompttemplate을 생성
base_chat_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

partial_chat_prompt= base_chat_prompt.partial(role = '주식투자')

In [80]:
# partial로 생성된 프롬프트에 질문만 채워 프롬프트 구성
sample_question = '현재 2025년 5월 시장 상황에서 삼성전자 주식 전망은?'
message = partial_chat_prompt.format_messages(question = sample_question)
print(message)

[SystemMessage(content='당신은 주식투자 분야의 전문 지식인 입니다. 가능한 자세히 답변해 주세요', additional_kwargs={}, response_metadata={}), HumanMessage(content='현재 2025년 5월 시장 상황에서 삼성전자 주식 전망은?', additional_kwargs={}, response_metadata={})]


In [ ]:
answer = (partial_chat_prompt | llm | parser).invoke({'question': sample_question})
print(answer) # 2025년 5월을 말하고 있지만 해당모델은 과거의 데이터로 학습한 모델로 신뢰할 수 없는 답변임

2025년 5월의 삼성전자 주식 전망을 평가하기 위해서는 여러 가지 요소를 고려해야 합니다. 다음은 삼성전자의 주식 전망에 영향을 미칠 수 있는 주요 요인들입니다.

1. **반도체 시장 동향**: 삼성전자는 세계 최대의 반도체 제조업체 중 하나입니다. 반도체 수요는 AI, 클라우드 컴퓨팅, IoT(사물인터넷) 등 다양한 산업의 성장에 따라 증가하고 있습니다. 2025년에는 이러한 기술들이 더욱 발전할 것으로 예상되므로, 반도체 수요가 지속적으로 증가할 가능성이 높습니다. 그러나 반도체 시장은 경기 사이클에 민감하므로, 글로벌 경제 상황에 따라 변동성이 클 수 있습니다.

2. **스마트폰 및 전자제품 판매**: 삼성전자는 스마트폰, 가전제품 등 다양한 전자제품을 제조하고 있습니다. 5G와 같은 새로운 기술의 도입이 소비자 수요에 긍정적인 영향을 미칠 수 있습니다. 또한, 새로운 제품 라인업이나 혁신적인 기술이 출시될 경우, 시장 점유율을 높일 수 있는 기회가 될 것입니다.

3. **글로벌 경제 상황**: 글로벌 경제의 성장률, 금리, 인플레이션 등은 삼성전자의 실적에 큰 영향을 미칠 수 있습니다. 특히, 미국과 중국 간의 무역 관계, 유럽의 경제 상황 등은 삼성전자의 수출에 영향을 미칠 수 있습니다.

4. **경쟁 상황**: 삼성전자는 애플, 화웨이, TSMC 등과 같은 강력한 경쟁자들과 경쟁하고 있습니다. 경쟁사의 기술 발전이나 가격 전략에 따라 삼성전자의 시장 점유율과 수익성이 영향을 받을 수 있습니다.

5. **ESG(환경, 사회, 지배구조) 경영**: 최근 투자자들은 ESG 요소를 중요하게 고려하고 있습니다. 삼성전자가 지속 가능한 경영을 통해 긍정적인 이미지를 구축하고, 투자자들의 신뢰를 얻는다면 주가에 긍정적인 영향을 미칠 수 있습니다.

6. **주가 평가**: 삼성전자의 주가는 PER(주가수익비율), PBR(주가순자산비율) 등 다양한 지표를 통해 평가할 수 있습니다. 현재 주가가 과대평가 또는 과소평가되어 있는지를 분석하는 것도 중요합니다

LangchainMemory 
```
이전 대화 내용을 기억해서 문맥을 유지하는 역활 LangChain 0, 3X부터는 LCEL 기반으로 체인을 구성
3X부터는 LCEL 기반으로 체인을 구성 , RunnableWithMessageHistory, ChatMessageHistory 등의 컴포넌트를 활용해서 세션별 대화기록을 관리, 대화가 장기화될 경우 요약 메모리를 도입해서 과거 대화를 LLM으로 요약하고 축약된 형태로 저장해서 프롬프트의 길이 문제를 해결
```

In [88]:
%pip install langchain_redis

   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------- ----------------------------- 3.4/12.6 MB 20.2 MB/s eta 0:00:01
   -------------------------- ------------- 8.4/12.6 MB 22.6 MB/s eta 0:00:01
   ----------------------------- ---------- 9.2/12.6 MB 14.3 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 16.8 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.3.0
    Uninstalling urllib3-2.3.0:
      Successfully uninstalled urllib3-2.3.0
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 3.1.5
    Uninstalling Jinja2-3.1.5:
   

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-common 1.2 requires numpy<2.1.4,>=1.25.0, but you have numpy 2.2.6 which is incompatible.
autogluon-core 1.2 requires numpy<2.1.4,>=1.25.0, but you have numpy 2.2.6 which is incompatible.
autogluon-features 1.2 requires numpy<2.1.4,>=1.25.0, but you have numpy 2.2.6 which is incompatible.
autogluon-multimodal 1.2 requires accelerate<1.0,>=0.34.0, but you have accelerate 1.7.0 which is incompatible.
autogluon-multimodal 1.2 requires numpy<2.1.4,>=1.25.0, but you have numpy 2.2.6 which is incompatible.
autogluon-multimodal 1.2 requires torch<2.6,>=2.2, but you have torch 2.7.0 which is incompatible.
autogluon-multimodal 1.2 requires torchvision<0.21.0,>=0.16.0, but you have torchvision 0.22.0 which is incompatible.
autogluon-tabular 

In [84]:
from dotenv import load_dotenv
load_dotenv()

True

In [85]:
from langchain_core.chat_history import InMemoryChatMessageHistory
# 메모리 객체 생성
history = InMemoryChatMessageHistory()
history.add_user_message('안녕하세요 제 이름은 홍길동 입니다')
history.add_ai_message('안녕하세요 홍길동님! 무엇을 도와드릴까요?')
# 현재까지의 대화 내용 확인
for msg in history.messages:
    print(f'{msg.type} : {msg.content}')

human : 안녕하세요 제 이름은 홍길동 입니다
ai : 안녕하세요 홍길동님! 무엇을 도와드릴까요?


In [ ]:
# Redis 기반 채팅 기록 저장소 (나중에 클라우드랑 같이)
from langchain_redis import RedisChatMessageHistory
import os
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
session_id = 'user_123'
history = RedisChatMessageHistory(session_id=session_id, redis_url=REDIS_URL)
history.add_user_message('안녕하세요 제 이름은 홍길동 입니다')
history.add_ai_message('안녕하세요 홍길동님! 무엇을 도와드릴까요?')
# 현재까지의 대화 내용 확인
for msg in history.messages:
    print(f'{msg.type} : {msg.content}')

In [130]:
# 세션기반 다중사용자 메모리 구조 구현 - 다중사용자 챗봇
# 핵심 : session_id를 키로 하는 메모리 저장소를 만들고 사용자의 대화는 키별로 저정한다.
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
# 프롬프트
prompt = ChatPromptTemplate.from_messages([
    ('system', '당신은 뛰어난 한구어 상담 챗봇입니다. 질문에 친절하고 자세히 답변해주세요'),
    MessagesPlaceholder(variable_name='history'),
    ('human','{input}')
])
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

In [131]:
# LCEL
from langchain_core.output_parsers import StrOutputParser
chain = prompt | llm | StrOutputParser()

In [132]:
# 세션별 메모리 저장소를 딕셔너리롤 만들고, 존재하지 않는 새로운 세션 id가 들어오면 InMemorryChatMessageHistory를 생성
# get_session_history를 구현

In [133]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
# 세션 id -> 대화 기록 객체 매핑
store = {}
def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    """
    세션 ID에 해당하는 대화 기록 객체를 반환 (없으면 새로 생성)
    Input:
        session
    Output:
        InMemoryChatMessageHistory
    """
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 메모리를 통합한 체인 래퍼 생성
chatbot = RunnableWithMessageHistory(    
    chain,
    get_session_history,    
    input_messages_key = 'input',
    history_messages_key='history'
)


In [134]:
# 두개의 세션을 번갈아가면서 대화  RunnableWithMessageHistory 가 각 세션에 맞는 대화 기록을 관리합니다.
sessions = ['user_a','user_b']
questions = [
    '안녕하세요, 저는 홍길동입니다. 당신은 누구신가요?',   # usre_a 첫번재 질문
    '안녕하세요, 저는 이순신입니다. 당신은 어떤 일을 하시나요?', # user_b 첫번째 질문
    '저는 프로그래밍을 배우고 있습니다. 당신은 어떤 일을 하시나요?', # user_a 두번째 질문
    '저는 역사에 관심이 많습니다. 당신은 어떤 분야에 관심이 있나요?' # user_b 두번째 질문
]
for i, question in enumerate(questions):
    session_id = sessions[i % 2]  # 세션 ID를 번갈아가며 사용
    result = chatbot.invoke({'input': question}, config={'configurable': {'session_id': session_id}})
    print(f'[{session_id}] 질문: {question}')
    print(f'[{session_id}] 챗봇: {result}\n')

16:46:20 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[user_a] 질문: 안녕하세요, 저는 홍길동입니다. 당신은 누구신가요?
[user_a] 챗봇: 안녕하세요, 홍길동님! 저는 여러분의 질문에 답변하고 도움을 드리기 위해 만들어진 AI 상담 챗봇입니다. 궁금한 점이나 도움이 필요한 부분이 있다면 언제든지 말씀해 주세요!

16:46:22 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[user_b] 질문: 안녕하세요, 저는 이순신입니다. 당신은 어떤 일을 하시나요?
[user_b] 챗봇: 안녕하세요, 이순신님! 저는 여러분의 질문에 답변하고, 다양한 주제에 대해 정보를 제공하는 상담 챗봇입니다. 역사, 과학, 문화, 일상생활 등 여러 분야에 대해 이야기할 수 있습니다. 궁금한 점이나 도움이 필요한 부분이 있다면 언제든지 말씀해 주세요!

16:46:26 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[user_a] 질문: 저는 프로그래밍을 배우고 있습니다. 당신은 어떤 일을 하시나요?
[user_a] 챗봇: 프로그래밍을 배우고 계시다니 멋지네요! 저는 주로 여러분의 질문에 답변하고, 정보나 조언을 제공하는 역할을 합니다. 프로그래밍 관련 질문이나 학습에 도움이 필요하시면 언제든지 도와드릴 수 있습니다. 어떤 언어를 배우고 계신가요? 또는 어떤 부분에서 도움이 필요하신가요?

16:46:28 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[user_b] 질문: 저는 역사에 관심이

In [136]:
result = chatbot.invoke({'input': "저는 철수에요, 반갑습니다."}, config={'configurable': {'session_id': 'user_c'}})
print(f'[user_c] 질문: 저는 철수에요, 반갑습니다.')
print(f'[user_c] 챗봇: {result}\n')

16:46:45 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[user_c] 질문: 저는 철수에요, 반갑습니다.
[user_c] 챗봇: 안녕하세요, 철수님! 반갑습니다. 어떻게 도와드릴까요? 궁금한 점이나 이야기하고 싶은 주제가 있다면 말씀해 주세요.



In [137]:
result = chatbot.invoke({'input': "저는 누구라구요?"}, config={'configurable': {'session_id': 'user_a'}})
result

16:46:54 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'홍길동님이라고 말씀하셨습니다! 혹시 더 알고 싶으신 내용이나 다른 질문이 있으신가요? 언제든지 말씀해 주세요!'

In [138]:
result = chatbot.invoke({'input': "저는 누구라구요?"}, config={'configurable': {'session_id': 'user_b'}})
result

16:47:00 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'이순신님이라고 말씀하셨습니다! 이순신은 조선시대의 유명한 장군으로, 임진왜란 동안의 뛰어난 군사 전략과 용맹으로 잘 알려져 있습니다. 혹시 이순신 장군에 대해 더 이야기하고 싶으신가요, 아니면 다른 역사적 인물이나 사건에 대해 궁금한 점이 있으신가요?'

In [139]:
result = chatbot.invoke({'input': "저는 누구라구요?"}, config={'configurable': {'session_id': 'user_c'}})
result

16:47:05 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'철수님이라고 말씀하셨습니다! 혹시 더 궁금한 점이나 다른 이야기를 나누고 싶으신가요? 언제든지 말씀해 주세요!'

요약 메모리 구현(대화내용 자동 요약)
```
긴 대화 내용을 모두 프롬프트에 기록하는 것은 비 효율적 -> 프롬프트의 길이 제한에 걸릴 가능성이 있음
Conversation Summary Memory
0.3x 버전에서는 직접 요약용 제안을 만들어서 ChatMessageHistory에 적용
```
어떻게 요약?
```
- 일정길이 이상으로 대화가 누적되면, 과거 대화를 요약해서 핵심내용만 담김
- 요약결과를 메모리에 시스템 메세지 등으로 저장 -> 메모리 절약
- 새로운 사용자 입력시 요약된 맥락 + 최근 및 메세지만 참고해서 llm 전달
```

In [140]:
# 요약용 프롬프트 템플릿
summary_prompt = ChatPromptTemplate.from_messages([
    ('system','당신은 대화 요약 전문가입니다. 대화의 주요 내용을 간결하게 요약해 주세요'),
    ('human','{conversation}') # 전체 대화내용을 하나의 문자열로 전달
])
# LCEL
summary_chain = summary_prompt | llm | StrOutputParser()

In [143]:
# user_d 세션에 대화 내용을 기록 긴 대화 생성
long_queries = [
    '안녕, 오늘 뭐하려고 했지',
    '아 맞다 내일 회의자료 준비해야지, 회의는 몇시지?',
    '그 회의에 누가 참석하는지 기억나?',
    '단위프로젝트 진행 상황도 공유해야 할까?',
    '최근에 이야기 했던 새로운 기능에 대한 업데이는 있어?'
]
session_id = 'user_d'
for q in long_queries:
    answer =  chatbot.invoke({'input': q}, config={'configurable': {'session_id': session_id}})
    # print(answer)

print(f'요약전 user_d의 메모리 메세지 개수 : {len(store[session_id].messages)}')


17:25:51 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
17:25:53 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
17:25:55 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
17:25:58 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
17:25:59 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
요약전 user_d의 메모리 메세지 개수 : 30


In [147]:
# 전체 대화 내용을 요약하고 마지막 사용처 질문-답변 쌍만 원본 유지
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
# 요약 대상 대화내용 추출(마지막 QA 쌍 제외한 이전 내용)
message = store[session_id].messages
if len(message) > 2:
   original_dialog = '\n'.join([f'{msg.type.upper()} : {msg.content}' for msg in message[:-2]])
else:
   original_dialog = '\n'.join([f'{msg.type.upper()} : {msg.content}' for msg in message])

# llm으로 요약 샘플
summary_text = summary_chain.invoke({'conversation':original_dialog})
print('== 요약 내용 ==')
print(summary_text)
# 기존 메모리를 요약으로교체 -> 이전 내용 요약본 +  최근 QA 유지
new_history = InMemoryChatMessageHistory()
new_history.messages.append(SystemMessage(content=f'요약:{summary_text}'))
# 최근 대화의 마지막 QA쌍을 복원
if len(message) >= 2:
    last_user_msg = message[-2]
    last_ai_msg = message[-1]

    if isinstance(last_user_msg, HumanMessage):
        new_history.add_user_message(last_user_msg.content)
    else:
        new_history.messages.append(last_user_msg)

    if isinstance(last_ai_msg, AIMessage):
        new_history.add_ai_message(last_ai_msg)
    else:
        new_history.messages.append(last_ai_msg)
      

== 요약 내용 ==
대화 요약:

- 인간은 오늘의 계획을 묻고, AI는 다양한 활동을 제안함.
- 인간은 내일 회의 자료 준비를 언급하며 회의 시간을 물음. AI는 이메일이나 캘린더에서 확인하라고 조언.
- 인간은 회의 참석자를 기억하지 못하고, AI는 초대장이나 팀원에게 문의하라고 안내.
- 인간은 단위 프로젝트 진행 상황 공유 여부를 질문하고, AI는 공유의 중요성을 강조하며 도움을 제안.
- 인간은 새로운 기능에 대한 업데이트를 묻고, AI는 관련 팀원에게 문의하라고 조언하며 회의에서 공유할 것을 추천.


In [148]:
len(store[session_id].messages)

30